In [2]:
# Imports

import os

import matplotlib.pyplot as plt
import matplotlib

# PyTorch Lightning
import pytorch_lightning as pl
import seaborn as sns

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

from tqdm.notebook import tqdm
from IPython.display import set_matplotlib_formats
from pytorch_lightning.callbacks import ModelCheckpoint

# Import GPU-related things
if torch.cuda.is_available():
    import cupy as np
    import cudf as pd
    
    # Ensure that all operations are deterministic on GPU (if used) for reproducibility
    torch.backends.cudnn.determinstic = True
    torch.backends.cudnn.benchmark = False
else:
    import numpy as np
    import pandas as pd
    
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

# Plotting
plt.set_cmap("cividis")
%matplotlib inline
set_matplotlib_formats("svg", "pdf")  # For export
matplotlib.rcParams["lines.linewidth"] = 2.0
sns.reset_orig()

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = os.environ.get("PATH_DATASETS", "data/")
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models/")

# Setting the seed
pl.seed_everything(42)

print('CUDA:', torch.cuda.is_available())
print("Device:", device)

/home/caigh/miniconda3/envs/mm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_18399/1334366771.py:40: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("svg", "pdf")  # For export
Global seed set to 42


CUDA: False
Device: cpu


In [8]:
files = [
    'data.csv',
    'edrug3d.sdf',
    'qm9-1.sdf',
    'qm9-2.sdf',
    'qm9-3.sdf',
    'qm9-4.sdf',
    'qm9-5.sdf',
    'qm9-6.sdf',
    'qm9-7.sdf',
    'qm9-8.sdf'
]


def check_missing_files():
    """Checks for missing files. Returns true, if all files are present."""
    for file in files:
        if not os.path.exists('./data/' + file):
            return False

    return True

if not check_missing_files():
    !wget -nc -O data.zip "https://hochschulebonnrheinsieg-my.sharepoint.com/:u:/g/personal/nico_piel_365h-brs_de1/ESuGOTn_IflEk7I5HkOFpbwBZKeOk9Qf2nL5JEcq2om6_Q?e=sHYsTk&download=1"
    !unzip -u data.zip
    !rm data.zip

--2022-05-25 09:25:45--  https://hochschulebonnrheinsieg-my.sharepoint.com/:u:/g/personal/nico_piel_365h-brs_de1/ESuGOTn_IflEk7I5HkOFpbwBZKeOk9Qf2nL5JEcq2om6_Q?e=sHYsTk&download=1
Resolving hochschulebonnrheinsieg-my.sharepoint.com (hochschulebonnrheinsieg-my.sharepoint.com)... 13.107.136.9, 13.107.138.9
Connecting to hochschulebonnrheinsieg-my.sharepoint.com (hochschulebonnrheinsieg-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/nico_piel_365h-brs_de1/Documents/Uni/Bachelor/data.zip?ga=1 [following]
--2022-05-25 09:25:45--  https://hochschulebonnrheinsieg-my.sharepoint.com/personal/nico_piel_365h-brs_de1/Documents/Uni/Bachelor/data.zip?ga=1
Reusing existing connection to hochschulebonnrheinsieg-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 47705176 (45M) [application/x-zip-compressed]
Saving to: ‘data.zip’

data.zip            100%[===================>]  45.50M  22.7MB/s    in 2.0

In [9]:
class CustomDataset(data.Dataset):
    def __init__(self, path):
        super().__init__()
        self.data = pd.read_csv(path)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        series = self.data.iloc[idx]
        return series[0], series[1]

In [10]:
dataset = CustomDataset('./data/data.csv')
dataloader = data.DataLoader(dataset, batch_size=64, shuffle=True)

In [11]:
class DataModule(pl.LightningDataModule):
    def __init__(self):
        self.train_dataset = CustomDataset('./data/data.csv')
        self.val_dataset_1 = CustomDataset('./data/data.csv')
        self.val_dataset_2 = CustomDataset('./data/data.csv')
        self.test_dataset = CustomDataset('./data/data.csv')
        self.predict_dataset = CustomDataset('./data/data.csv')

    def train_dataloader(self):
        return data.DataLoader(self.train_dataset, batch_size=64, shuffle=True)

    def val_dataloader(self):
        return data.DataLoader(self.val_dataset_1)

    def test_dataloader(self):
        return data.DataLoader(self.test_dataset)

    def predict_dataloader(self):
        return data.DataLoader(self.predict_dataset)

In [ ]:
class ATTransformer(pl.LightningModule):
    def __init__():
        super().__init__()
        self.save_hyperparameters()
        self._create_model()

    def _create_model(self):
        raise NotImplementedError

    def forward(self, X):
